In [175]:
import json
import numpy as np
import pandas as pd
import glob, os
import re
from typing import List

In [176]:
# load recipes 
# load recipes file 
recipes_ds = pd.read_csv("/home/victor/Documents/Expectation_data_generation/src/recipes/processed_recipes_dataset_id.csv", 
                         index_col=0,
                         sep="|")

In [177]:
# start text processing 
cwd = os.getcwd()

In [178]:
cwd

'/home/victor/Documents/Expectation_data_generation/src/meals_collection'

In [179]:
files_to_process = list(glob.glob("extended_*.json"))
len(files_to_process)

13

In [180]:
files_to_process[-1]

'extended_recipes_raw_4000_4500.json'

In [181]:
data_loaded = None
with open(files_to_process[0], "r") as fp:
    data_loaded = json.load(fp)
len(data_loaded.keys())

1923

In [182]:
def extract_ingredients(text: str):
    extracted_text = ""
    text_temp = text.lower()
    try:
        # Define start and end strings for extraction
        start_str = "ingredients"
        end_str = "preparation"

        # Use regular expression to extract text between start and end strings
        res = re.search(re.escape(start_str) + "(.*?)" + re.escape(end_str), text_temp, re.DOTALL)

        if res:
            extracted_text = res.group(1).strip()
            #print(extracted_text)
        else:
            print("Start and end strings not found in the text.")
        # temp_text = raw_text.lower()
        # start_str = "ingredients"
        # end_str = "preparation"
        # s = str(re.escape(start_str))
        # e = str(re.escape(end_str))
        # res = re.findall(s+"(.*)"+e,temp_text)[0]
    except Exception as e:
        print(f"Error ingredients: {e}")
    return extracted_text
    

In [183]:
def extract_preparation_steps(text:str):
    temp_text = text.lower()
    preparation = ""
    str_1 = "preparation steps"
    str_2 = "preparation"
    try:
        test_position = temp_text.find(str_1)
        if test_position == -1:
            test_position = temp_text.find(str_2)
            if test_position == -1:
                print("Preparation not found")
                return preparation
            else:
                preparation = temp_text.split(str_2)[-1]
        else:
            preparation = temp_text.split(str_1)[-1]
    except Exception as e:
        print(f"Error preparation steps: {e}")
    return preparation

In [184]:
def generated_updated_dataset(file_list:List):
    final_df = pd.DataFrame(columns=["title", "ingredients", "preparation"])
    list_row = ["", "", ""]
    for f in files_to_process:
        # loading data 
        print(f"processing file: {f}")
        data_loaded = None
        with open(f, "r") as fp:
            data_loaded = json.load(fp)
        if data_loaded:
            for k, v in data_loaded.items():
                list_row = ["", "", ""]
                list_row[0] = k
                try:
                    list_row[1] = extract_ingredients(v).replace(":", "")
                    if len(list_row[1]) == 0:
                        print(k)
                    list_row[2] = extract_preparation_steps(v).replace(":", "")
                except Exception as e:
                    print(f"Error: {e} key: {k}")
                final_df.loc[len(final_df)] = list_row
    return final_df

In [185]:
final_df = generated_updated_dataset(files_to_process)

processing file: extended_recipes_raw_second_group_1500_3500.json
Start and end strings not found in the text.
 Guacamole and Salsa with Tortilla Chips
    - Calories per portion
Start and end strings not found in the text.
 BBQ pulled chicken sandwiches
- Calories
Preparation not found
Start and end strings not found in the text.
 Broccoli and cheddar stuffed sweet potatoes
- Calories
Preparation not found
Start and end strings not found in the text.
 Vegan Apple Crumble
    Calories
Preparation not found
Start and end strings not found in the text.
 Stuffed Bell Peppers with Rice and Beans
    Calories
Preparation not found
Start and end strings not found in the text.
 Baked Eggplant Fries
    Calories
Preparation not found
Start and end strings not found in the text.
 Cinnamon Sugar Bagel
Calories
Start and end strings not found in the text.
 Oatmeal with Blueberries and Almonds
   - Calories
Preparation not found
Start and end strings not found in the text.
 Cucumber and Tomato San

In [186]:
final_df.shape

(8451, 3)

In [187]:
final_df.tail(4)

,title,ingredients,preparation
8447,Israeli Eggplant Salad - 300 calories per por...,\n- 2 medium-sized eggplants\n- 2 tablespoons ...,\n1. preheat the oven to 400°f (200°c). \n2. w...
8448,Chinese General Tso's Chicken - 400 calories ...,"\n- 1 lb boneless, skinless chicken breasts, c...","\n1. in a mixing bowl, toss the chicken pieces..."
8449,Mexican Shrimp Tacos - 350 calories per porti...,"\n- 1 lb (450g) shrimp, peeled and deveined\n-...","\n1. in a large bowl, toss the shrimp with oli..."
8450,Thai Pad Thai Noodles - 400 calories per port...,,


In [188]:
# fusion dataframes 
extended_df = recipes_ds.merge(final_df, on='title')

In [189]:
extended_df.tail(4)

,title,raw_text,meal_type,cultural_restriction,calories,allergies,recipeId,ingredients,preparation
8447,Caprese quinoa salad with grilled chicken - 3...,97. Caprese quinoa salad with grilled chicken ...,dinner,NaN,350.0,contains lactose,food_8447,\n- 1 cup quinoa\n- 2 cups water\n- 2 chicken ...,"\n1. in a saucepan, bring the water to a boil ..."
8448,Baked coconut shrimp with mango salsa and jas...,98. Baked coconut shrimp with mango salsa and ...,dinner,NaN,420.0,contains shellfish,food_8448,"\n- 10 large shrimp, peeled and deveined\n- 1/...",\n1. preheat the oven to 425°f (220°c).\n\n2. ...
8449,One-pot creamy chicken and mushroom pasta - 4...,99. One-pot creamy chicken and mushroom pasta ...,dinner,NaN,480.0,contains gluten,food_8449,\n- 2 tablespoons olive oil\n- 1 pound chicken...,"\n1. in a large pot or dutch oven, heat the ol..."
8450,"Stuffed bell peppers with couscous, chickpeas...","100. Stuffed bell peppers with couscous, chick...",dinner,NaN,350.0,contains lactose,food_8450,\n- 4 bell peppers (any color)\n- 1 cup cousco...,\n1. preheat your oven to 375°f (190°c).\n2. c...


In [190]:
extended_df.shape

(8451, 9)

In [193]:
extended_df["title"] = extended_df["title"].apply(lambda x: transform_title(x))

In [192]:
# fix titles 
def transform_title(text:str):
    new_text = text.strip()
    if "-" in text:
        new_text = text.split("-")[0]
    return new_text

In [194]:
import re
def extract_calories(raw_text: str):
    # normalize text 
    temp_text = raw_text.lower()
    pattern = r'calories\s*per\s*portion[:]*\s*(\d+)\s*kcal'
    matches = re.findall(pattern, temp_text)
    if matches:
        numbers = [int(match) for match in matches]
        return numbers[0]
    else:
        return None

In [195]:
# fix the problem on calories 
mask = extended_df["calories"] < 0

In [196]:
# solve calories problems
rows_problem = extended_df.loc[mask, :]

In [197]:
extended_df.loc[mask, "calories"] = extended_df.loc[mask, "raw_text"].apply(lambda x: extract_calories(x))

In [198]:
extended_df["calories"]

0        70.0
1       150.0
2       200.0
3       250.0
4       300.0
        ...  
8446    450.0
8447    350.0
8448    420.0
8449    480.0
8450    350.0
Name: calories, Length: 8451, dtype: float64

In [199]:
extended_df.to_csv("extended_recipes_partial.csv", sep="|", index=False)

In [200]:
extended_df.columns

Index(['title', 'raw_text', 'meal_type', 'cultural_restriction', 'calories',
       'allergies', 'recipeId', 'ingredients', 'preparation'],
      dtype='object')

In [330]:
extended_df["calories"]

0        70.0
1       150.0
2       200.0
3       250.0
4       300.0
        ...  
8446    450.0
8447    350.0
8448    420.0
8449    480.0
8450    350.0
Name: calories, Length: 8451, dtype: float64

## Process additional data 

In [201]:
base_path = "/home/victor/Documents/Expectation_data_generation/"

In [236]:
list_files_to_process = glob.glob(os.path.join(base_path, 
                                               "Additional_info_recipes_1000_1200_*.json"))
print(f"files to process: {len(list_files_to_process)}")

files to process: 4


In [238]:
# load one file to check 
data_loaded = None 
with open(list_files_to_process[0], "r") as fp:
    data_loaded = json.load(fp)

In [239]:
data_loaded["Mexican Bean Burrito Bowl\nTotal calories per portion"]

"To calculate the nutritional information per portion of the Mexican Bean Burrito Bowl, we need to gather the nutritional values of each ingredient. Here are the approximate values per cup or measurement:\n\n- 1 cup cooked brown rice:\n    - Carbohydrates: 45g\n    - Protein: 4.5g\n    - Fat: 1.75g\n    - Fiber: 3.5g\n    - Calories: 216\n\n- 1 cup canned black beans:\n    - Carbohydrates: 40g\n    - Protein: 15g\n    - Fat: 0.5g\n    - Fiber: 15g\n    - Calories: 227\n\n- 1 cup corn kernels (fresh or canned):\n    - Carbohydrates: 24g\n    - Protein: 4.5g\n    - Fat: 2.5g\n    - Fiber: 3g\n    - Calories: 132\n\n- 1 cup diced tomatoes:\n    - Carbohydrates: 7g\n    - Protein: 1.5g\n    - Fat: 0.5g\n    - Fiber: 2g\n    - Calories: 32\n\n- 1/2 cup diced red onion:\n    - Carbohydrates: 6g\n    - Protein: 0.5g\n    - Fat: 0g\n    - Fiber: 1g\n    - Calories: 25\n\n- 1/2 cup diced bell peppers (any color):\n    - Carbohydrates: 3.5g\n    - Protein: 0.5g\n    - Fat: 0g\n    - Fiber: 1g\n 

In [209]:
data_loaded['Vegan Quinoa and Black Bean Chili\nTotal calories per portion']

"Here is the breakdown of the nutritional content per portion of Vegan Quinoa and Black Bean Chili:\n\n- Carbohydrates: Quinoa (1 cup) - approximately 39g\n  Black beans (15 ounces) - approximately 36g\n  Diced tomatoes (15 ounces) - approximately 14g\n  Red bell pepper - approximately 9g\n\n- Protein: Quinoa (1 cup) - approximately 8g\n  Black beans (15 ounces) - approximately 15g\n\n- Fat: Olive oil (1 tablespoon) - approximately 14g\n\n- Fiber: Quinoa (1 cup) - approximately 5g\n  Black beans (15 ounces) - approximately 15g\n\n- Calories: Quinoa (1 cup) - approximately 222 calories\n  Olive oil (1 tablespoon) - approximately 120 calories\n  Onion, garlic, bell pepper - minimal calorie content\n  Black beans (15 ounces) - approximately 220 calories\n  Diced tomatoes (15 ounces) - approximately 70 calories\n  Chopped green chilies (4 ounces) - minimal calorie content\n  Vegetable broth (2 cups) - minimal calorie content\n  Chili powder, cumin, paprika, oregano, salt, and pepper - mini

In [240]:
import re

def extract_nutritional_values(text):
    # Define dictionaries to store the extracted values
    nutritional_values = {
        'Carbohydrates': [],
        'Protein': [],
        'Fat': [],
        'Fiber': []
    }
    
    # Define regular expressions for each nutritional component
    components = ['Carbohydrates', 'Protein', 'Fat', 'Fiber']
    for component in components:
        regex = rf'{component}:(.*?)\n'
        matches = re.findall(regex, text, re.DOTALL)
        
        for match in matches:
            lines = match.strip().split('\n')
            for line in lines:
                nutritional_values[component].append(line)
    
    return nutritional_values

In [241]:
def extract_nutritional_values_from_files(file_list:List[str]):
    answer_dict = {}
    for f in file_list:
        print(f"Processing file: {f}")
        # load the file 
        data_loaded = None 
        with open(f, "r") as fp:
            data_loaded = json.load(fp)
        if data_loaded:
            # process files
            temp_dict = {}
            for k in data_loaded.keys():
                try:
                    temp_dict[k] = extract_nutritional_values(data_loaded[k])
                except Exception as e:
                    print(f"Exception processing: {k}")
                    continue
            answer_dict.update(temp_dict)
        else:
            print(f"File: {f} has a loading error")
    return answer_dict        
        

In [242]:
nutritional_values_dict = extract_nutritional_values_from_files(list_files_to_process)

Processing file: /home/victor/Documents/Expectation_data_generation/Additional_info_recipes_1000_1200_1100.json
Processing file: /home/victor/Documents/Expectation_data_generation/Additional_info_recipes_1000_1200_1050.json
Processing file: /home/victor/Documents/Expectation_data_generation/Additional_info_recipes_1000_1200_1000.json
Processing file: /home/victor/Documents/Expectation_data_generation/Additional_info_recipes_1000_1200_1150.json


In [244]:
# filter dictionaries 
new_dict = {}
components = ['Carbohydrates', 'Protein', 'Fat', 'Fiber']
for k in nutritional_values_dict.keys():
    temp_dict = nutritional_values_dict[k]
    conserve = True
    for comp in components:
        if len(temp_dict[comp]) < 1:
            conserve = conserve and False
    if conserve:
        new_dict[k] = nutritional_values_dict[k]

In [245]:
len(new_dict)

181

In [256]:
import re

# Sample text containing floating-point numbers
text = "The price of the product is $19.99, and the weight is 2.5 kg."

# Use regular expression to find floating-point numbers
float_numbers = re.findall(r"[-+]?\d*\.\d+|\d+", text)

# Convert the matched strings to floating-point numbers
float_numbers = [float(number) for number in float_numbers]

# Print the extracted floating-point numbers
print(float_numbers)

[19.99, 2.5]


In [262]:
# Generate total values 
data_frame = pd.DataFrame(columns=['title', 
                                   'Carbohydrates', 
                                   'Protein', 
                                   'Fat', 
                                   'Fiber'])
for k in new_dict.keys():
    print(f"Processing {k}")
    components = ['Carbohydrates', 'Protein', 'Fat', 'Fiber']
    list_row = [k, "", "", "", ""]
    for idx, comp in enumerate(components):
        # check if the total is at the end 
        temp_list = []
        for item in new_dict[k][comp]:
            # Use regular expression to find floating-point numbers
            float_numbers = re.findall(r"[-+]?\d*\.\d+|\d+", item)
            # Convert the matched strings to floating-point numbers
            float_numbers = [float(number) for number in float_numbers] 
            if len(float_numbers) > 0:
                result = max(float_numbers)
            else:
                result = 0.0
            temp_list.append(result)
        total = sum(temp_list)
        last_value = temp_list[-1]
        total_w = total - last_value
        closeness = np.isclose(total_w, last_value, atol=3)
        if closeness:
            list_row[idx+1] = last_value
        else:
            list_row[idx+1] = total
    data_frame.loc[len(data_frame)] = list_row

Processing Mexican Bean Burrito Bowl
Total calories per portion
Processing Vegan Quinoa and Black Bean Chili
Total calories per portion
Processing Italian Minestrone Soup
Total calories per portion
Processing Vegan Spinach and Artichoke Dip
Total calories per portion
Processing Greek Moussaka
Total calories per portion
Processing Vegan Gnocchi with Pesto Sauce
Total calories per portion
Processing Korean Bibimbap
Total calories per portion
Processing Moroccan Chickpea Stew
Total calories per portion
Processing Vegan Pho
Total calories per portion
Processing Spanish Gazpacho
Total calories per portion
Processing Vegan Spinach and Mushroom Quiche
Total calories per portion
Processing Thai Mango Salad
Total calories per portion
Processing Vegan Stuffed Cabbage Rolls
Total calories per portion
Processing Lebanese Fattoush Salad
Total calories per portion
Processing Vegan Lentil Sloppy Joes
Total calories per portion
Processing Vegan Ratatouille Pasta
Total calories per portion
Processing T

In [263]:
data_frame

,title,Carbohydrates,Protein,Fat,Fiber
0,Mexican Bean Burrito Bowl\nTotal calories per ...,150.00,28.00,26.25,38.5
1,Vegan Quinoa and Black Bean Chili\nTotal calor...,39.00,8.00,14.00,5.0
2,Italian Minestrone Soup\nTotal calories per po...,138.00,32.00,30.00,28.0
3,Vegan Spinach and Artichoke Dip\nTotal calorie...,11.00,7.00,16.00,3.0
4,Greek Moussaka\nTotal calories per portion,140.00,103.00,120.00,18.0
...,...,...,...,...,...
176,Vegan Cauliflower Wings,196.00,37.00,5.40,20.0
177,Veggie Frittata,33.64,65.52,64.88,8.0
178,Lentil and Spinach Pasta,86.00,31.00,30.00,20.0
179,Vegan Goulash,138.00,25.00,2.00,38.0


In [264]:
# cross with the other dataset 


In [266]:
final_test = extended_df.merge(data_frame, on="title")

In [268]:
final_test.head(4)

,title,raw_text,meal_type,cultural_restriction,calories,allergies,recipeId,ingredients,preparation,Carbohydrates,Protein,Fat,Fiber
0,Vegan Coconut Chia Pudding,31. Recipe: Vegan Coconut Chia Pudding\nCalori...,afternoon snacks,vegan,NaN,NaN,food_1084,\n- 1/4 cup chia seeds\n- 1 cup coconut milk\n...,"\n1. in a mixing bowl, combine chia seeds, coc...",28.0,5.0,21.0,10.0
1,Vegan Guacamole,32. Recipe: Vegan Guacamole\nCalories per port...,afternoon snacks,vegan,NaN,NaN,food_1085,"\n- 2 ripe avocados\n- 1 small onion, finely c...","\n1. cut the avocados in half, remove the pit,...",42.0,4.0,29.0,17.0
2,Vegan Spinach Dip,33. Recipe: Vegan Spinach Dip\nCalories per po...,afternoon snacks,vegan,NaN,NaN,food_1086,"\n- 1 cup raw cashews\n- 1 cup frozen spinach,...",\n1. soak the cashews in hot water for 1 hour....,24.0,13.0,24.0,6.0
3,Vegan Tofu Stir,34. Recipe: Vegan Tofu Stir-Fry\nCalories per ...,afternoon snacks,vegan,NaN,NaN,food_1087,\n- 1 block of firm tofu\n- 2 tablespoons soy ...,\n1. press the tofu to remove excess moisture....,71.0,25.0,44.5,13.0


In [271]:
mask = final_test["calories"].isna()

In [272]:
# to fix 
to_fix_df = final_test.loc[mask, :]

In [285]:
to_fix_df["raw_text"][20]

'57. Recipe: Vegan Mushroom and Spinach Quesadillas\nCalories per portion: 300\nTotal Portions: 1\nAllergic Warnings: Contains soy\nOrigin Country: N/A\n\n'

In [321]:
def extract_calories(text):
    raw_text = text.lower()
    lines = raw_text.split('\n')
    calories = -1
    for line in lines:
        if "calories per portion:" in line:
            l = line.split("calories per portion:")
            for sl in l:
                if len(sl) > 0:
                    sl = sl.strip()
                    if sl.isdigit():
                        calories = int(sl)
        elif "calories:" in line:
            l = line.split("calories:")
            for sl in l:
                if len(sl) > 0:
                    integers_kcal = re.findall(r'(\d+)', text)
                    integers_kcal = [int(number) for number in integers_kcal]
                    if len(integers_kcal) == 1:
                        calories = integers_kcal[0]
                    elif len(integers_kcal) > 1:
                        calories = max(integers_kcal)
                    else:
                        calories = -1   
        elif "kcal" in line:
            integers_kcal = re.findall(r'(\d+)\s*kcal', text)
            integers_kcal = [int(number) for number in integers_kcal]
            if len(integers_kcal) == 1:
                calories = integers_kcal[0]
            elif len(integers_kcal) > 1:
                calories = max(integers_kcal)
            else:
                calories = -1
        else:
            pass
    return calories
        

In [324]:
final_test.loc[mask, "calories"] = to_fix_df["raw_text"].apply(lambda x: extract_calories(x))

In [326]:
final_test.to_csv("extended_recipes.csv", sep="|", index=False)

In [307]:
"hola que tal".split('-')

['hola que tal']

In [323]:
text = to_fix_df.loc[tl == -1, "raw_text"]

In [311]:
text = text.lower()

In [314]:
list_test = text.split('\n')

In [319]:
if "calories:" in list_test[0]:
    numbers = list_test[0].split("calories:")
    print(numbers)
    print(True)

['12. vegetable stir-fry - ', ' 200, portions: 2, allergenic warnings: none, origin: -']
True


In [320]:
import re

# Sample text
text = ' 200, portions: 2, allergenic warnings: none, origin: -'

# Use regular expressions to extract numbers
numbers = re.findall(r'\d+', text)

# Convert the matched strings to integers
numbers = [int(number) for number in numbers]

# Print the extracted numbers
print(numbers)

[200, 2]


In [248]:
np.isclose(10, 6, atol=5)

True

In [246]:
new_dict

{'Mexican Bean Burrito Bowl\nTotal calories per portion': {'Carbohydrates': ['45g',
   '40g',
   '24g',
   '7g',
   '6g',
   '3.5g',
   '1g',
   '12g',
   '11g',
   '150g'],
  'Protein': ['4.5g',
   '15g',
   '4.5g',
   '1.5g',
   '0.5g',
   '0.5g',
   '0g',
   '2g',
   '0g',
   '28g'],
  'Fat': ['1.75g',
   '0.5g',
   '2.5g',
   '0.5g',
   '0g',
   '0g',
   '0g',
   '21g',
   '0g',
   '26.25g'],
  'Fiber': ['3.5g',
   '15g',
   '3g',
   '2g',
   '1g',
   '1g',
   '0g',
   '10g',
   '2g',
   '38.5g']},
 'Vegan Quinoa and Black Bean Chili\nTotal calories per portion': {'Carbohydrates': ['Quinoa (1 cup) - approximately 39g'],
  'Protein': ['Quinoa (1 cup) - approximately 8g'],
  'Fat': ['Olive oil (1 tablespoon) - approximately 14g'],
  'Fiber': ['Quinoa (1 cup) - approximately 5g']},
 'Italian Minestrone Soup\nTotal calories per portion': {'Carbohydrates': ['0g',
   '10g',
   '14g',
   '3g',
   '2g',
   '16g',
   '4g',
   '0g',
   '35g',
   '8g',
   '4g',
   '40g',
   '1g',
   '1g'],
  

In [243]:
nutritional_values_dict

{'Mexican Bean Burrito Bowl\nTotal calories per portion': {'Carbohydrates': ['45g',
   '40g',
   '24g',
   '7g',
   '6g',
   '3.5g',
   '1g',
   '12g',
   '11g',
   '150g'],
  'Protein': ['4.5g',
   '15g',
   '4.5g',
   '1.5g',
   '0.5g',
   '0.5g',
   '0g',
   '2g',
   '0g',
   '28g'],
  'Fat': ['1.75g',
   '0.5g',
   '2.5g',
   '0.5g',
   '0g',
   '0g',
   '0g',
   '21g',
   '0g',
   '26.25g'],
  'Fiber': ['3.5g',
   '15g',
   '3g',
   '2g',
   '1g',
   '1g',
   '0g',
   '10g',
   '2g',
   '38.5g']},
 'Vegan Quinoa and Black Bean Chili\nTotal calories per portion': {'Carbohydrates': ['Quinoa (1 cup) - approximately 39g'],
  'Protein': ['Quinoa (1 cup) - approximately 8g'],
  'Fat': ['Olive oil (1 tablespoon) - approximately 14g'],
  'Fiber': ['Quinoa (1 cup) - approximately 5g']},
 'Italian Minestrone Soup\nTotal calories per portion': {'Carbohydrates': ['0g',
   '10g',
   '14g',
   '3g',
   '2g',
   '16g',
   '4g',
   '0g',
   '35g',
   '8g',
   '4g',
   '40g',
   '1g',
   '1g'],
  

In [215]:
# Sample text
text = data_loaded["Mexican Bean Burrito Bowl\nTotal calories per portion"]

# Extract nutritional values
nutritional_values = extract_nutritional_values(text)

# Print the extracted values
for component, values in nutritional_values.items():
    print(component)
    for item, value in values.items():
        print(f"{item}: {value}")
    print()

Carbohydrates

Protein

Fat

Fiber

